# WMS-Master
Controls WMS-Random and WMS-Smart

In [12]:
import os
import numpy as np
from matplotlib import pyplot as plt
from colorama import Fore, Style, init
import csv
import statistics
import math

In [13]:
from classes.CellIterator import CellIterator
from classes.Item import Item
from classes.Worker import Worker
from classes.Warehouse import Warehouse

In [14]:
NUM_BAYS = 6
RACK_LENGTH = 15
CELLS_PER_BAY = 20
CELL_LENGTH = RACK_LENGTH / CELLS_PER_BAY
RACK_WIDTH = 0.8
BAY_WIDTH = RACK_WIDTH / 2
LEVEL_HEIGHT = 0.2

In [15]:
WHR = Warehouse(
    NUM_BAYS,
    RACK_LENGTH,
    CELLS_PER_BAY,
    BAY_WIDTH,
    LEVEL_HEIGHT
)

WHS = Warehouse(
    NUM_BAYS,
    RACK_LENGTH,
    CELLS_PER_BAY,
    BAY_WIDTH,
    LEVEL_HEIGHT
)

In [16]:
def margin_of_error(std_dev, sample_size, z_score=1.96):
    return z_score * (std_dev / math.sqrt(sample_size))

In [17]:
def log_total_cells_to_csv(costs, file_path):
    # Ensure the directory exists
    os.makedirs(os.path.dirname(file_path), exist_ok=True)

    # Check if the file exists
    file_exists = os.path.isfile(file_path)
    
    # Open the file in append mode
    with open(file_path, mode='a', newline='') as file:
        writer = csv.writer(file)
        
        # If the file doesn't exist, write the header
        if not file_exists:
            headers = ['n'] + [f'cst{i+1}' for i in range(len(costs))] + ['avg', 'min', 'max', 'std', 'moe']
            writer.writerow(headers)

        # Get the trial number
        if file_exists:
            # Read the existing data to determine the trial number
            with open(file_path, mode='r') as read_file:
                reader = csv.reader(read_file)
                rows = list(reader)
                trial_number = len(rows)  # Assuming the header is included, this will give the next trial number
        else:
            trial_number = 1

        # Calculate average, minimum, maximum, and variance
        avg_cost = sum(costs) / len(costs)
        min_cost = min(costs)
        max_cost = max(costs)
        std_cost = round(math.sqrt(statistics.variance(costs)) if len(costs) > 1 else 0, 2)
        moe_cost = round(margin_of_error(std_cost, len(costs)), 2)

        # Write the new record
        writer.writerow([trial_number] + costs + [avg_cost, min_cost, max_cost, std_cost, moe_cost])

def get_average_total_cost(file_path='results/wms.csv'):
    # Check if the file exists
    if not os.path.isfile(file_path):
        print(f"File {file_path} does not exist.")
        return None

    total_cost = 0
    num_trials = 0

    with open(file_path, mode='r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            total_cost += int(row['cost'])
            num_trials += 1

    if num_trials == 0:
        return 0  # Avoid division by zero if no records are found

    average_cost = total_cost / num_trials
    return average_cost

In [18]:
def wms_runner(WH, type):
    WH.generate_item_list_small("./data/order_history_small.csv")
    
    # create a warehouse configuration (random / smart)
    if type == "random": WH.sprinkle_aisle()
    elif type == "smart": WH.smart_sprinkle()

    directory = 'data/orders'
    costs = []

    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        if not os.path.isfile(file_path): continue

        WH.read_orders(file_path)
        WH.dist_batched_order_list()

        for worker in WH.workers: worker.min_max_order_list(WH.aisle_items)

        total_cells = 0
        for worker in WH.workers:
            worker.fulfill_batched_order()
            total_cells += worker.cells_travelled

        costs.append(total_cells)
        WH.reset("soft")

    if type == "random": log_total_cells_to_csv(costs, "results/wms-random.csv")
    if type == "smart": log_total_cells_to_csv(costs, "results/wms-smart.csv")

    WH.reset("hard")
    # print("Done!")

In [19]:
wms_random_path = "results/wms-random.csv"
if os.path.exists(wms_random_path):
    os.remove(wms_random_path)
    print(f"{wms_random_path} has been deleted successfully.")
else:
    print(f"{wms_random_path} does not exist.")

N = 1
for _ in range(N):
    wms_runner(WHR, type="random")
    wms_runner(WHS, type="smart")
# wms_runner(WHS, type="smart")

results/wms-random.csv has been deleted successfully.


# Flask

In [24]:
WHR.cell_capacity_cm3

600000.0000000001